## Crime Data import

In [2]:
# Import data

ny_crimes_df = pd.read_csv('data/NYPD_Complaint_Data_Historic.csv')

In [ ]:
# Get random selection from data

ny_crimes_df = ny_crimes_df.sample(frac=0.10).reset_index(drop=True)

In [3]:
# Select only desirable columns

# Reshape crime data - get only desired columns

ny_crimes_df = ny_crimes_df[['CMPLNT_NUM', 'KY_CD', 'OFNS_DESC', 'Latitude', 'Longitude']] 

# Replace column names

temp_cols = ['complaint_id', 'event_code', 'event_description', 'lat', 'lng']
ny_crimes_df.columns = temp_cols

ny_crimes_df.head()


In [ ]:
# Repicate function for assigning neighborhood by lat/lng

from geopy import distance
import pandas as pd

# Requires importing neighborhood data

neighborhood_df = pd.read_csv('data/100_venues_for_all_ny_hoods.csv')

def get_closest_neighborhood(lat, lng):
    temp_df = neighborhood_df[['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude']]
    distance_list = []

    # loop through neighborhoods and get distance
    for n, n_lat, n_lng in zip(temp_df['Neighborhood'], temp_df['Neighborhood Latitude'], temp_df['Neighborhood Longitude']):
        distance_list.append(distance.distance(
            (lat, lng),
            (n_lat, n_lng)
        ).km)

    # Use index of minimum distance to get neighborhood name 
    return temp_df['Neighborhood'][distance_list.index(min(distance_list))]


## Assign neighborhoods for each incident

In [ ]:
# Crimes

stop = ny_crimes_df.shape[0]
updates = range(0, stop, 500)

neighborhoods = []
for lat, lng in zip(ny_crimes_df['lat'], ny_crimes_df['lng']):
    try:
        n = get_closest_neighborhood(lat, lng)
    except:
        n = np.nan
    neighborhoods.append(n)
    for i in updates:
        if len(neighborhoods) == i:
            print(f"Matched {i} events.")

print(neighborhoods[0])

ny_crimes_df['neighborhood'] = neighborhoods

ny_crimes_df.head()

In [ ]:
# save data

ny_crimes_df.to_csv('data/parsed_crimes_with_neighborhoods.csv')